<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/notebooks/draft/football_EDA_Graph_dataset_preparation_220918.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Downloads

### Installations

In [ ]:
#!pip install chart_studio

### Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)

from glob import glob
from tqdm import tqdm
import pickle
import functools

1.3.5


### Code

#### Load Dataset in DataFrame

In [ ]:
%%time
start_date = '2008-01-01'
end_date = '2022-09-05'
data_csv_list = ['./calcio/results_221002/' + str(dd).replace('-', '') + '.csv' 
                 for dd in pd.date_range(start=start_date, end=end_date).date]
data_df = pd.concat(map(functools.partial(pd.read_csv, sep=';', compression=None),data_csv_list), ignore_index = True)

CPU times: user 32.6 s, sys: 2.81 s, total: 35.5 s
Wall time: 35.4 s


In [ ]:
data_df.Id[data_df.Id == '4xtqmstjixo8392sildh47g9g'] = [id + str(cnt) for cnt, id in enumerate(data_df.Id[data_df.Id == '4xtqmstjixo8392sildh47g9g'])]
data_df.Id[data_df.Id == 'eaib9fp2qivl5t5jf5u5h3v9w'] = [id + str(cnt) for cnt, id in enumerate(data_df.Id[data_df.Id == 'eaib9fp2qivl5t5jf5u5h3v9w'])]

In [ ]:
data_df['timestamp'] = pd.to_datetime(data_df['BeginTime'], dayfirst = True).astype('int64') // 10**9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.


In [ ]:
data_df[['date', 'times_ext']] = data_df['BeginTime'].str.split(expand = True)
data_df = data_df.drop(['BeginTime', 'times_ext'], axis = 'columns')

#### Refactoring DataFrame

In [ ]:
data_df['sum_score'] = data_df['Result1'] + data_df['Result2'] # Сумма голов в матче
# Приводим все счета к сумме мячей в матче не больше 10 (adj)
data_df['sum_score_k'] = [1 if score_k < 11 else score_k / 10 for score_k in data_df['sum_score']]
data_df['home_score_adj'] = (data_df['Result1'] / data_df['sum_score_k']).astype(int)
data_df['away_score_adj'] = (data_df['Result2'] / data_df['sum_score_k']).astype(int)
data_df['score_adj'] = data_df['home_score_adj'].astype(str) + '-' + data_df['away_score_adj'].astype(str)

### Code_dataset refactoring

#### Refactoring DataFrame

In [ ]:
winners_list = []
for hm, aw in tqdm(data_df[['Result1', 'Result2']].values):
    if hm > aw:
        winners_list.append('H')
    elif hm < aw:
        winners_list.append('A')
    else:
        winners_list.append('D')
data_df['Winner'] = winners_list

100%|██████████| 2682527/2682527 [00:03<00:00, 705480.36it/s]


In [ ]:
data_df.HomeId = data_df.HomeId.astype(int)
data_df.AwayId = data_df.AwayId.astype(int)

#### Расчитываем время прошедшее от прошлого матча и делаем словарик связей матчей

In [ ]:
#data_df['date'] = pd.to_datetime(data_df['date'])
data_df = data_df.sort_values(by = 'timestamp').reset_index(drop = True)
timestamp7days = 7 * 24 * 60 * 60

In [ ]:
team_GId_dict = {}
zero_diff = 0
for info in tqdm(
    zip(
        data_df.timestamp,
        data_df.HomeId,
        data_df.AwayId,
        data_df.Id,
    ),
    total=len(data_df),
                    ):
    time_stamp = info[0]
    homeID = info[1]
    awayID = info[2]
    matchID = info[3]
    # 1. Проверить если ID команды в словаре, если нет перейти к добавлению
    if homeID in team_GId_dict:
        # 2. Проверить время, если время позднее последнего добаления,
        # то можно просто присоединить снизу, инфо о последнем матче команды,
        # иначе перейти во вставку матча между матчами
        if matchID not in team_GId_dict[homeID]:
            if time_stamp >= team_GId_dict[homeID]["last_time"]:
                # 3. Добавление матча для команды в словарь
                previous_num = team_GId_dict[homeID]["last_index"]
                previous_time = team_GId_dict[homeID]["last_time"]
                # 4. Обновление блока последнего матча для команды
                team_GId_dict[homeID]["last_index"] = matchID
                team_GId_dict[homeID]["last_time"] = time_stamp
                # 3. Добавление матча для команды в словарь
                team_GId_dict[homeID].update(
                    {
                        matchID: [
                            previous_num,
                            time_stamp,
                        ]
                    }
                )
            else:
                zero_diff += 1
                # 5. Поиск точки вхождение для матча, который оказался не новым
                current_index = team_GId_dict[homeID]["last_index"]
                previous_match_time = team_GId_dict[homeID][current_index][1]
                previous_index = team_GId_dict[homeID][current_index][0]
                while (info[0] < previous_match_time) & (previous_index != -1):
                    current_index = previous_index
                    previous_index = team_GId_dict[homeID][previous_index][0]
                    previous_match_time = team_GId_dict[homeID][current_index][1]
                # 6. Вставка матча и обновление соседних 2 матчей
                team_GId_dict[homeID].update(
                    {matchID: [team_GId_dict[homeID][current_index][0], time_stamp]}
                )
                team_GId_dict[homeID].update(
                    {current_index: [matchID, time_stamp]}
                )

    else:
        # 3. Добавление матча для команды в словарь. Новая команда
        team_GId_dict[homeID] = {"last_index": info[3]}
        team_GId_dict[homeID].update({"last_time": time_stamp})
        team_GId_dict[homeID].update(
            {
                matchID: [
                    -1,
                    time_stamp - timestamp7days
                ]
            }
        )

    #############################################################################
    #############################################################################

    if awayID in team_GId_dict:
        # 2. Проверить время, если время позднее последнего добаления,
        # то можно просто присоединить снизу, инфо о последнем матче команды,
        # иначе перейти во вставку матча между матчами
        if matchID not in team_GId_dict[awayID]:
            if time_stamp >= team_GId_dict[awayID]["last_time"]:
                # 3. Добавление матча для команды в словарь
                previous_num = team_GId_dict[awayID]["last_index"]
                previous_time = team_GId_dict[awayID]["last_time"]
                # 4. Обновление блока последнего матча для команды
                team_GId_dict[awayID]["last_index"] = matchID
                team_GId_dict[awayID]["last_time"] = time_stamp
                # 3. Добавление матча для команды в словарь
                team_GId_dict[awayID].update(
                    {
                        matchID: [
                            previous_num,
                            time_stamp,
                        ]
                    }
                )
            else:
                zero_diff += 1
                # 5. Поиск точки вхождение для матча, который оказался не новым
                current_index = team_GId_dict[awayID]["last_index"]
                previous_match_time = team_GId_dict[awayID][current_index][1]
                previous_index = team_GId_dict[awayID][current_index][0]
                while (time_stamp < previous_match_time) & (previous_index != -1):
                    current_index = previous_index
                    previous_index = team_GId_dict[awayID][previous_index][0]
                    previous_match_time = team_GId_dict[awayID][current_index][1]
                # 6. Вставка матча и обновление соседних 2 матчей
                team_GId_dict[awayID].update(
                    {matchID: [team_GId_dict[awayID][current_index][0], time_stamp]}
                )
                team_GId_dict[awayID].update(
                    {current_index: [matchID, time_stamp]}
                )

    else:
        # 3. Добавление матча для команды в словарь. Новая команда
        team_GId_dict[awayID] = {"last_index": info[3]}
        team_GId_dict[awayID].update({"last_time": time_stamp})
        team_GId_dict[awayID].update(
            {
                matchID: [
                    -1,
                    time_stamp - timestamp7days
                ]
            }
        )


100%|██████████| 2682527/2682527 [00:28<00:00, 94979.29it/s]


#### Рассчитываем паузу между матчами

In [ ]:
data_df['team_rest_home'] = [team_GId_dict[team][idx][1] - team_GId_dict[team][team_GId_dict[team][idx][0]][1]
                             if team_GId_dict[team][idx][0] != -1 else timestamp7days
                             for team, idx in tqdm(zip(data_df.HomeId, data_df.Id), total = len(data_df))]
data_df['team_rest_away'] = [team_GId_dict[team][idx][1] - team_GId_dict[team][team_GId_dict[team][idx][0]][1]
                             if team_GId_dict[team][idx][0] != -1 else timestamp7days
                             for team, idx in tqdm(zip(data_df.AwayId, data_df.Id), total = len(data_df))]

100%|██████████| 2682527/2682527 [00:08<00:00, 306756.08it/s]


In [ ]:
# Отдых команд разделяем на 3 группы
data_df['team_rest_home_adj'] = [0 if tm < 500_000 else 1 if tm < 1_000_000 else 2 for tm in data_df['team_rest_home']]
data_df['team_rest_away_adj'] = [0 if tm < 500_000 else 1 if tm < 1_000_000 else 2 for tm in data_df['team_rest_away']]

####Создание токенов-фичей score-home_away-пауза

In [ ]:
match_cat_dict = {}
cnt = 1

for home_score in range(11):
    for away_score in range(11):
        for home_place in range(2):
            for rest_time in range(3):
                for curr_home in range(2):
                    if (home_score + away_score) < 11:
                        match_cat_dict[
                            str(home_score) + '-' + \
                            str(away_score) + ':' + \
                            str(home_place) + ':' + \
                            str(rest_time) + ':' + \
                            str(curr_home)
                                        ] = cnt
                        cnt += 1

none_idx = 0 #max(match_cat_dict.values())

print(match_cat_dict)
print(len(match_cat_dict))
print('none_idx: ', none_idx)

{'0-0:0:0:0': 1, '0-0:0:0:1': 2, '0-0:0:1:0': 3, '0-0:0:1:1': 4, '0-0:0:2:0': 5, '0-0:0:2:1': 6, '0-0:1:0:0': 7, '0-0:1:0:1': 8, '0-0:1:1:0': 9, '0-0:1:1:1': 10, '0-0:1:2:0': 11, '0-0:1:2:1': 12, '0-1:0:0:0': 13, '0-1:0:0:1': 14, '0-1:0:1:0': 15, '0-1:0:1:1': 16, '0-1:0:2:0': 17, '0-1:0:2:1': 18, '0-1:1:0:0': 19, '0-1:1:0:1': 20, '0-1:1:1:0': 21, '0-1:1:1:1': 22, '0-1:1:2:0': 23, '0-1:1:2:1': 24, '0-2:0:0:0': 25, '0-2:0:0:1': 26, '0-2:0:1:0': 27, '0-2:0:1:1': 28, '0-2:0:2:0': 29, '0-2:0:2:1': 30, '0-2:1:0:0': 31, '0-2:1:0:1': 32, '0-2:1:1:0': 33, '0-2:1:1:1': 34, '0-2:1:2:0': 35, '0-2:1:2:1': 36, '0-3:0:0:0': 37, '0-3:0:0:1': 38, '0-3:0:1:0': 39, '0-3:0:1:1': 40, '0-3:0:2:0': 41, '0-3:0:2:1': 42, '0-3:1:0:0': 43, '0-3:1:0:1': 44, '0-3:1:1:0': 45, '0-3:1:1:1': 46, '0-3:1:2:0': 47, '0-3:1:2:1': 48, '0-4:0:0:0': 49, '0-4:0:0:1': 50, '0-4:0:1:0': 51, '0-4:0:1:1': 52, '0-4:0:2:0': 53, '0-4:0:2:1': 54, '0-4:1:0:0': 55, '0-4:1:0:1': 56, '0-4:1:1:0': 57, '0-4:1:1:1': 58, '0-4:1:2:0': 59, '0-4:

In [ ]:
input_list = []
for info in tqdm(data_df['score_adj'], total = len(data_df)):
    cat_key = info + ':' + '0'
    input_list.append(cat_key)
data_df['home_idx'] = input_list

100%|██████████| 2682527/2682527 [00:01<00:00, 1445353.45it/s]


In [ ]:
input_list = []
for info in tqdm(data_df['score_adj'], total = len(data_df)):
    cat_key = info + ':' + '1'
    input_list.append(cat_key)
data_df['away_idx'] = input_list

100%|██████████| 2682527/2682527 [00:01<00:00, 1411118.92it/s]


In [ ]:
for info in tqdm(zip(data_df.HomeId, data_df.AwayId, data_df.Id, data_df.home_idx, data_df.away_idx), total = len(data_df)):
    homeID = info[0]
    awayID = info[1]
    matchID = info[2]
    homeIDX = info[3]
    awayIDX = info[4]
    team_GId_dict[homeID][matchID] += [homeIDX]
    team_GId_dict[awayID][matchID] += [awayIDX]

100%|██████████| 2682527/2682527 [00:12<00:00, 221125.18it/s]


#### Шифтуем токен на один уровень вверх

In [ ]:
input_list = []
for gid, haid, rest in tqdm(
    zip(data_df.Id.values, data_df.HomeId, data_df.team_rest_home_adj),
    total = len(data_df)):
    prev_id = team_GId_dict[haid][gid][0]
    if prev_id == -1:
        input_list.append(none_idx)
    else:
        key = team_GId_dict[haid][prev_id][2] + ':' + \
        str(rest) + ':' + '0'
        input_list.append(match_cat_dict[key])
data_df['home_token'] = input_list

100%|██████████| 2682527/2682527 [00:10<00:00, 259709.63it/s]


In [ ]:
input_list = []
for gid, haid, rest in tqdm(
    zip(data_df.Id.values, data_df.AwayId, data_df.team_rest_away_adj),
    total = len(data_df)):
    prev_id = team_GId_dict[haid][gid][0]
    if prev_id == -1:
        input_list.append(none_idx)
    else:
        key = team_GId_dict[haid][prev_id][2] + ':' + \
        str(rest) + ':' + '1' 
        input_list.append(match_cat_dict[key])
data_df['away_token'] = input_list

100%|██████████| 2682527/2682527 [00:10<00:00, 258148.34it/s]


In [ ]:
#node_df_columns = ['id'] + [str(x) for x in range(none_idx + 1)]
#node_df = pd.DataFrame(columns = node_df_columns)

####Создаем файлы вершин, рёбер и конверсий

In [ ]:
id_dict = {}
#cnt = len(data_df) - 1 #обратный порядок
cnt = 0
for id, hid, aid, ts in tqdm(zip(
    data_df['Id'].values, 
    data_df['HomeId'].values, 
    data_df['AwayId'].values,
    data_df['timestamp'].values
                                ), total = len(data_df)):
    hidx = str(hid) + '_' + id
    aidx = str(aid) + '_' + id
    id_dict[hidx] = [cnt * 2, ts]
    id_dict[aidx] = [cnt * 2 + 1, ts]
    #cnt -= 1 #обратный порядок
    cnt +=1

100%|██████████| 2682527/2682527 [00:19<00:00, 137893.44it/s]


In [ ]:
edges_df = pd.DataFrame(columns = ['source', 'target'])

In [ ]:
#обратный порядок
#edges_df['source'] = [edge * 2 for edge in range(len(data_df) - 1, -1, -1)] #data_df['HomeId'].astype(str) + '_' + data_df['Id']
#edges_df['target'] = [edge * 2 + 1 for edge in range(len(data_df) - 1, -1, -1)] #data_df['AwayId'].astype(str) + '_' + data_df['Id']
#прямой порядок
edges_df['source'] = [edge * 2 for edge in range(len(data_df))]
edges_df['target'] = [edge * 2 + 1 for edge in range(len(data_df))]

In [ ]:
match_conection_list = []
for team, team_dict in tqdm(team_GId_dict.items(), total = len(team_GId_dict)):
    #print(team)
    #print(team_dict)
    last_index = team_dict['last_index']
    cnt = 0
    while team_dict[last_index][0] != -1:
        #print(str(team) + '_' + team_dict[last_index][0] + '_:_' +  str(team) + '_' + last_index)
        source_team = str(team) + '_' + team_dict[last_index][0]
        target_team = str(team) + '_' + last_index
        source_team_idx = id_dict[source_team][0]
        target_team_idx = id_dict[target_team][0]
        match_conection_list.append([source_team_idx, target_team_idx])
        #match_conection_list.append([target_team_idx, source_team_idx])
        last_index = team_dict[last_index][0]
        cnt +=1
        if cnt > 1_100:
            print('Cycled_graph')
            print('Team: ', team)

100%|██████████| 49761/49761 [00:27<00:00, 1801.24it/s] 


In [ ]:
pd.concat([
    edges_df, 
    pd.DataFrame(np.flip(edges_df.values, axis = 1), columns = ['source', 'target']),
    pd.DataFrame(match_conection_list, columns = ['source', 'target'])
    ]).to_csv('edges.csv.gz', index = False, compression ={'method':'gzip'})

In [ ]:
data_df['binary_output'] = 0
data_df['binary_output'][data_df['Winner'] == 'H'] = 1
data_df['class_H'] = 0
data_df['class_D'] = 0
data_df['class_A'] = 0
data_df['class_H'][data_df['Winner'] == 'H'] = 1
data_df['class_D'][data_df['Winner'] == 'D'] = 1
data_df['class_A'][data_df['Winner'] == 'A'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value

In [ ]:
data_df

,Id,Home,Away,League,Country,Type,Result1,Result2,Periods,HomeId,AwayId,LeagueId,Season,Round,timestamp,date,sum_score,sum_score_k,home_score_adj,away_score_adj,score_adj,Winner,team_rest_home,team_rest_away,team_rest_home_adj,team_rest_away_adj,home_idx,away_idx,home_token,away_token,binary_output,class_H,class_D,class_A
0,2rxecbv07olakwt1zbh52dihh,Al Zawra'a,Al Sinaah,iraq - iraqi league,iraq,NaN,1,0,NaN,3848,8382,r5750,2007-2008,group-stage,1199152800,01.01.2008,1,1.0,1,0,1-0,H,604800,604800,1,1,1-0:0,1-0:1,0,0,1,1,0,0
1,4l0rplzz4qo3zinjb6umuetw5,Bnot Tivon,Hapoel Petach Tikva,israel - ligat al women,israel,w,0,0,NaN,10064,10063,r5721,2007-2008,regular-season,1199152800,01.01.2008,0,1.0,0,0,0-0,D,604800,604800,1,1,0-0:0,0-0:1,0,0,0,0,1,0
2,4kr1zx14kau8doo5fgkh2dc5x,Maccabi Kishronot Hadera,ASA Tel Aviv,israel - ligat al women,israel,w,0,3,NaN,10065,10059,r5721,2007-2008,regular-season,1199152800,01.01.2008,3,1.0,0,3,0-3,A,604800,604800,1,1,0-3:0,0-3:1,0,0,0,0,0,1
3,4kiofd3uilp1k12wrmblq2o5x,Hapoel Rishon LeZion,Ironi Ramat HaSharon,israel - ligat al women,israel,w,0,5,NaN,10061,23303,r5721,2007-2008,regular-season,1199152800,01.01.2008,5,1.0,0,5,0-5,A,604800,604800,1,1,0-5:0,0-5:1,0,0,0,0,0,1
4,4k9mtsryqbsxeydo4h5sm97md,Maccabi Be'er Sheva Li,Bnot Sakhnin,israel - ligat al women,israel,w,4,0,NaN,10060,10057,r5721,2007-2008,regular-season,1199152800,01.01.2008,4,1.0,4,0,4-0,H,604800,604800,1,1,4-0:0,4-0:1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2682522,9llqpuftwy5k4f20kwi7v2cd0,Zabbar St. Patrick,Kalkara,malta - national amateur league,malta,NaN,1,0,NaN,3056,22648,r68592,20222023,regular-season,1662416100,05.09.2022,1,1.0,1,0,1-0,H,787500,778500,1,1,1-0:0,1-0:1,57,256,1,1,0,0
2682523,5rfb10yome9157s22g4cax2qc,FAR Rabat,RSB Berkane,morocco - botola pro,morocco,NaN,1,1,NaN,3463,7512,r71423,20222023,regular-season,1662417000,05.09.2022,2,1.0,1,1,1-1,D,263700,345600,0,0,1-1:0,1-1:1,19,14,0,0,1,0
2682524,rmc4gmil9oi8fn8cvh5f8vtg,CDC Manicoré,Atlético Amazonense,brazil - amazonense women,brazil,w,2,2,NaN,57046,59567,r67381,2022,1st-phase,1662418800,05.09.2022,4,1.0,2,2,2-2,D,781200,802800,1,1,2-2:0,2-2:1,765,40,0,0,1,0
2682525,bfc93t0bmbrpxsan16rjzitqs,Chaves,Rio Ave,portugal - primeira liga,portugal,NaN,1,1,NaN,1704,1683,r69322,20222023,regular-season,1662419700,05.09.2022,2,1.0,1,1,1-1,D,780300,693900,1,1,1-1:0,1-1:1,33,376,0,0,1,0


In [ ]:
#cnt = 0
#cnt = len(data_df) - 1 #обратный порядок
cnt = 0
node_list = []
for htoken, atoken, bo, h, d, a, id in tqdm(zip(
    data_df['home_token'], 
    data_df['away_token'],
    data_df['binary_output'],
    data_df['class_H'],
    data_df['class_D'],
    data_df['class_A'],
    data_df['Id']
                                ), total = len(data_df)):
    node_list.append([cnt * 2, htoken, bo, h, d, a, id])
    node_list.append([cnt * 2 + 1, atoken, int(not(bo)), a, d, h, id])
    #cnt -= 1 #обратны порядок
    cnt +=1
pd.DataFrame(node_list).to_csv('nodes.csv.gz', index = False, compression ={'method':'gzip'})

100%|██████████| 2682527/2682527 [00:12<00:00, 213322.44it/s]


In [ ]:
pd.DataFrame.from_dict(
            id_dict, 
            orient = 'index', 
            columns = ['token', 'timestamp']
                        ).reset_index().to_csv(
                'conversion.csv.gz', 
                index = False, 
                compression ={'method':'gzip'}
                                                )

####Выгружаем в Нептун

In [ ]:
!pip install neptune-client

In [ ]:
import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('cred_andrey.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [ ]:
data_version = 'data_graph_221019/'
username, api_key = get_credential()
upload = True
if upload:
    project = neptune.init_project(
        name="scomesse/football", 
        api_token = api_key
        )
    
    #project[data_version + 'nodes'].upload('./nodes.csv.gz')
    project[data_version + 'edges_rev'].upload('./edges_rev.csv.gz')
    #project[data_version + 'conversion'].upload('./conversion.csv.gz')

    project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


####snap-stanford

In [ ]:
#!pip install snap-stanford

In [ ]:
import snap
S = snap.TNGraph.New()

In [ ]:
nodes_qty = 500_000
for node in range(nodes_qty):
    S.AddNode(node)

In [ ]:
edges_df = pd.read_csv('./edges.csv.gz')

In [ ]:
for row in tqdm(edges_df.values, total = len(edges_df)):
    if S.IsNode(int(row[0])) & S.IsNode(int(row[1])):
        if not(S.IsEdge(int(row[0]), int(row[1]))):
            S.AddEdge(int(row[0]), int(row[1]))

In [ ]:
%%time
snap.PrintInfo(S, 'Commutes', 'output.txt', False)
for line in open('output.txt').readlines():
    print (line.strip())

####spektral

In [ ]:
#!pip install spektral

In [ ]:
import pandas as pd
from sys import getsizeof
import numpy as np
from tqdm import tqdm
import gc
from scipy import sparse as sp

In [ ]:
from spektral.data import Dataset, Graph

In [ ]:
class calcio_graph(Dataset):
    def __init__(self, amount = 500_000, train = 0.5, validation = 0.3, **kwargs):
        self.amount = amount
        self.train = train
        self.validation = validation
        self.mask_tr = self.mask_va = self.mask_te = None
        super().__init__(**kwargs)
    def read(self):
        x_df = pd.read_csv('./nodes.csv.gz', usecols = [0, 1, 2], header = 0, names = ['idx','feat', 'y'])
        x_df = x_df.iloc[-self.amount:, :].set_index('idx').sort_index()
        x = np.zeros((len(x_df), max(x_df.feat) + 1))
        for cnt, feat in enumerate(x_df.feat):
            x[cnt, feat] = 1
        edges_df = pd.read_csv('./edges.csv.gz')
        edges_df = edges_df[(edges_df.source < self.amount) & (edges_df.target < self.amount)]
        a = sp.csr_matrix((
            np.ones(len(edges_df)), 
            (edges_df.source.values, edges_df.target.values)
                            ), shape=None).astype(np.float32)
        y = x_df.y.values
        self.mask_tr = np.arange(len(x_df)) < int(len(x_df) * self.train)
        self.mask_va = (np.arange(len(x_df)) > int(len(x_df) * self.train)) & \
                        (np.arange(len(x_df)) < int(len(x_df) * (self.train + self.validation)))
        self.mask_te = np.arange(len(x_df)) > int(len(x_df) * (self.train + self.validation))
        return [Graph(x = x, a = a, y = y)]

In [ ]:
dataset = calcio_graph(
                    #transforms=[NormalizeAdj()]
                    )

In [ ]:
dataset[0].y

array([1, 0, 0, ..., 1, 1, 0])

In [ ]:
(np.arange(10) > 4) & (np.arange(10) < 8) 

array([False, False, False, False, False,  True,  True,  True, False,
       False])

100%|██████████| 10680345/10680345 [00:21<00:00, 504042.08it/s]


In [ ]:
x_df = pd.read_csv('./nodes.csv.gz', usecols = [0, 1], header = 0, names = ['idx','feat'])

In [ ]:
x_df = x_df.iloc[-500_000:, :].set_index('idx').sort_index()

In [ ]:
x_df

,feat
idx,
0,381
1,136
2,33
3,376
4,765
...,...
499995,138
499996,691
499997,14


In [ ]:
x_arr = np.zeros((len(x_df), max(x_df.feat) + 1))

In [ ]:
x_arr.shape

(500000, 793)

In [ ]:
for cnt, feat in enumerate(x_df.feat):
    x_arr[cnt, feat] = 1

In [ ]:
edges_df = pd.read_csv('./edges.csv.gz')
edges_df = edges_df[(edges_df.source < 500_000) & (edges_df.target < 500_000)]

In [ ]:
adj = np.ones(len(edges_df))

In [ ]:
a = sp.csr_matrix((adj, (edges_df.source.values, edges_df.target.values)), shape=None).astype(np.float32)

In [ ]:
csr_index = np.arange(len(edges_df))

In [ ]:
a_idx = sp.csr_matrix((csr_index, (edges_df.source.values, edges_df.target.values)), shape=None).astype(np.float32)

In [ ]:
cx = sp.coo_matrix(a_idx)

In [ ]:
cx.row

array([     0,      1,      2, ..., 499998, 499999, 499999], dtype=int32)

In [ ]:
cx.col

array([     1,      0,      3, ..., 499999, 458776, 499998], dtype=int32)

In [ ]:
for i, j, v in zip(cx.row, cx.col, cx.data):
            index_list.append(int(v))

In [ ]:
#del data_df
gc.collect()

44

In [ ]:
round(getsizeof(data_df) / 1024 / 1024, 2)

3002.32